In [1]:
# --- Imports ---
import os, json, math, io, zipfile, time, re, shutil, glob, pathlib, sys, subprocess, shlex, tempfile, warnings
from pathlib import Path
from datetime import datetime as dt, timedelta, timezone, date, datetime
from dateutil import parser as dateparser
from dateutil.relativedelta import relativedelta
from urllib.parse import urljoin, quote
from functools import reduce

import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import pytz
import xarray as xr
import rasterio
from rasterio.mask import mask

import shapely
from shapely import ops
from shapely.geometry import Point, Polygon, box, mapping
from shapely.ops import unary_union, transform as shp_transform

from pyproj import Transformer
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import folium
import ee  # Earth Engine
from ee import EEException

from sklearn.neighbors import BallTree
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist
from geopy.distance import geodesic

import yaml

warnings.filterwarnings("ignore", category=UserWarning)


def skip_if_exists(path: str) -> bool:
    return os.path.exists(path)


In [2]:
# --- Configuration loader ---
from pathlib import Path
import os


def load_env_file(path: Path) -> dict:
    env = {}
    if not path.exists():
        return env
    for line in path.read_text().splitlines():
        line = line.strip()
        if not line or line.startswith("#") or "=" not in line:
            continue
        key, val = line.split("=", 1)
        env[key.strip()] = val.strip().strip('"').strip("'")
    return env


def apply_env_overrides() -> None:
    env = load_env_file(Path(".env"))
    for k, v in env.items():
        if v:
            os.environ[k] = v


# Fail fast unless PROJECT_DIR is explicitly provided.
apply_env_overrides()

PROJECT_DIR = os.environ.get("PROJECT_DIR")
if not PROJECT_DIR:
    raise FileNotFoundError(
        "PROJECT_DIR is not set. Add PROJECT_DIR to .env or environment variables."
    )

PROJECT_DIR = Path(PROJECT_DIR).expanduser().resolve()
CONFIG_PATH = PROJECT_DIR / "config" / "config.yaml"
if not CONFIG_PATH.exists():
    raise FileNotFoundError(f"Missing config file: {CONFIG_PATH}")

CONFIG = yaml.safe_load(CONFIG_PATH.read_text()) or {}


def resolve_out_path(path_str: str) -> str:
    p = Path(path_str)
    if p.is_absolute():
        return str(p)
    return str((Path(CONFIG["out_dir"]) / p).resolve())


SECRET_KEYS = [
    "CDS_UID",
    "CDS_API_KEY",
    "CDO_TOKEN",
    "AIRNOW_API_KEY",
    "PURPLEAIR_API_KEY",
    "ACS_KEY",
    "AQS_EMAIL",
    "AQS_KEY",
]
for key in SECRET_KEYS:
    env_val = os.environ.get(key)
    if env_val:
        CONFIG[key] = env_val

# Optional overrides for non-secret values via env vars.
for key in ["PURPLEAIR_SENSOR_INDEX"]:
    env_val = os.environ.get(key)
    if env_val:
        CONFIG[key] = env_val

# --- Local file structure ---
CONFIG["out_dir"] = str((PROJECT_DIR / CONFIG.get("out_dir", "results")).resolve())
CONFIG["NLCD_MANUAL_ZIP"] = str((Path(CONFIG["out_dir"]) / "manual" / "Annual_NLCD_LndCov_2024_CU_C1V1.zip").resolve())

os.makedirs(Path(CONFIG["out_dir"]) / "manual", exist_ok=True)

# EPSG constants (configurable)
WGS84_EPSG = int(CONFIG.get("WGS84_EPSG", 4326))
CA_ALBERS_EPSG = int(CONFIG.get("CA_ALBERS_EPSG", 3310))
OPS_EPSG = int(CONFIG.get("OPS_EPSG", CA_ALBERS_EPSG))

# Set working directory
os.chdir(PROJECT_DIR)

print(f"Config loaded from {CONFIG_PATH}")


Config loaded from /Users/Shared/blueleaflabs/hydropulse/config/config.yaml


In [3]:
# --- Ensure California boundary and build 3 km grid clipped to land ---


# Config
res_m = int(CONFIG.get("grid_resolution_m", 3000))
out_epsg = int(CONFIG.get("crs_epsg", 4326))
out_dir = CONFIG["out_dir"]; os.makedirs(out_dir, exist_ok=True)
inset_buffer_m = int(CONFIG.get("coast_inset_m", 0))  # e.g. 5000
boundary_path = CONFIG.get("ca_boundary_path", None)

# 1) Ensure boundary: download Census cartographic boundary if missing
if not boundary_path or not os.path.exists(boundary_path):
    states_zip = os.path.join(out_dir, "cb_2023_us_state_20m.zip")
    if not os.path.exists(states_zip):
        url = CONFIG["CENSUS_STATES_ZIP_URL"]
        r = requests.get(url, timeout=int(CONFIG.get("CENSUS_STATES_TIMEOUT", 120))); r.raise_for_status()
        with open(states_zip, "wb") as f: f.write(r.content)
    # Read from zip directly and select California
    states = gpd.read_file(f"zip://{states_zip}")
    if states.empty:
        raise ValueError("Census states file loaded empty.")
    ca = states[states["STATEFP"].astype(str).str.zfill(2).eq("06")][["geometry"]]
    if ca.empty:
        raise ValueError("California polygon not found in Census states file.")
    boundary_path = os.path.join(out_dir, "california_boundary.gpkg")
    ca.to_file(boundary_path, driver="GPKG")
    CONFIG["ca_boundary_path"] = boundary_path  # persist for later cells

# 2) Load boundary, dissolve, project, optional inward buffer
b = gpd.read_file(boundary_path)
if b.crs is None: raise ValueError("Boundary file has no CRS.")
b = b[["geometry"]].copy()
b = b.to_crs(CA_ALBERS_EPSG)
b = gpd.GeoDataFrame(geometry=[b.unary_union], crs=f"EPSG:{CA_ALBERS_EPSG}")
if inset_buffer_m > 0:
    b.geometry = b.buffer(-inset_buffer_m)
    b = gpd.GeoDataFrame(geometry=[b.unary_union], crs=f"EPSG:{CA_ALBERS_EPSG}")

# 3) Build snapped rectilinear grid over boundary bounds in EPSG:3310
minx, miny, maxx, maxy = b.total_bounds
snap_down = lambda v, s: np.floor(v/s)*s
snap_up   = lambda v, s: np.ceil(v/s)*s
minx, miny = snap_down(minx, res_m), snap_down(miny, res_m)
maxx, maxy = snap_up(maxx, res_m), snap_up(maxy, res_m)

xs = np.arange(minx, maxx, res_m)
ys = np.arange(miny, maxy, res_m)
n_rect = len(xs)*len(ys)
if n_rect > 3_500_000:
    raise MemoryError(f"Grid too large ({n_rect:,}). Increase res_m or tile the state.")

cells, col_i, row_j = [], [], []
for j, y in enumerate(ys):
    for i, x in enumerate(xs):
        cells.append(box(x, y, x+res_m, y+res_m)); col_i.append(i); row_j.append(j)

gdf_proj = gpd.GeoDataFrame({"col_i": np.int32(col_i), "row_j": np.int32(row_j)},
                            geometry=cells, crs=f"EPSG:{CA_ALBERS_EPSG}")
gdf_proj["cell_area_m2"] = float(res_m)*float(res_m)
gdf_proj["grid_id"] = f"CA3310_{res_m}_" + gdf_proj["col_i"].astype(str) + "_" + gdf_proj["row_j"].astype(str)

# 4) Strict land clip and land fraction
gdf_proj = gpd.sjoin(gdf_proj, b, how="inner", predicate="intersects").drop(columns=["index_right"])
inter = gpd.overlay(gdf_proj[["grid_id","geometry"]], b, how="intersection", keep_geom_type=True)
inter["land_area_m2"] = inter.geometry.area
land = inter[["grid_id","land_area_m2"]].groupby("grid_id", as_index=False).sum()
gdf_proj = gdf_proj.merge(land, on="grid_id", how="left")
gdf_proj["land_area_m2"] = gdf_proj["land_area_m2"].fillna(0.0)
gdf_proj["land_frac"] = (gdf_proj["land_area_m2"] / gdf_proj["cell_area_m2"]).clip(0,1)
gdf_proj = gdf_proj[gdf_proj["land_frac"] > 0].reset_index(drop=True)

# 5) Reproject to requested output CRS and save
grid_gdf = gdf_proj.to_crs(out_epsg)

parquet_path = os.path.join(out_dir, f"grid_{res_m}m_CA.parquet")
grid_gdf.to_parquet(parquet_path, index=False)

geojson_path = os.path.join(out_dir, f"grid_{res_m}m_CA_head10.geojson")
grid_gdf.head(10).to_file(geojson_path, driver="GeoJSON")

# Diagnostics
cell_area_km2 = (res_m/1000.0)**2
eff_land_km2 = float((grid_gdf.get("land_frac",1.0) * cell_area_km2).sum())
print(f"Saved: {parquet_path}")
print(f"Cells: {len(grid_gdf):,}")
print(f"Effective land area ≈ {round(eff_land_km2):,} km²")
print(f"Implied cell size ≈ {round((eff_land_km2/len(grid_gdf))**0.5,2)} km")

grid_gdf.head()


/var/folders/98/ykbpmmjd0pdf8zgd8bc1hhmr0000gn/T/ipykernel_37376/3235801746.py:34: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  b = gpd.GeoDataFrame(geometry=[b.unary_union], crs=f"EPSG:{CA_ALBERS_EPSG}")


Saved: /Users/Shared/blueleaflabs/hydropulse/results/grid_3000m_CA.parquet
Cells: 46,495
Effective land area ≈ 410,516 km²
Implied cell size ≈ 2.97 km


,col_i,row_j,geometry,cell_area_m2,grid_id,land_area_m2,land_frac
0,215,0,"POLYGON ((-117.09883 32.52004, -117.09786 32.5...",9000000.0,CA3310_3000_215_0,3.527256e+06,0.391917
1,216,0,"POLYGON ((-117.06697 32.51921, -117.06599 32.5...",9000000.0,CA3310_3000_216_0,2.924116e+06,0.324902
2,217,0,"POLYGON ((-117.03511 32.51837, -117.03411 32.5...",9000000.0,CA3310_3000_217_0,1.712565e+06,0.190285
3,218,0,"POLYGON ((-117.00325 32.51751, -117.00224 32.5...",9000000.0,CA3310_3000_218_0,5.055427e+05,0.056171
4,214,1,"POLYGON ((-117.12973 32.54795, -117.12876 32.5...",9000000.0,CA3310_3000_214_1,9.690513e+04,0.010767


In [4]:
# --- Persist config + save grid (3310 ops copy, 4326 preview) + write metadata ---

# Inputs assumed from prior cell:
# - grid_gdf            : current grid GeoDataFrame (any CRS)
# - CONFIG              : dict with out_dir, grid_resolution_m, crs_epsg, ca_boundary_path
# - CA_ALBERS_EPSG=3310 : defined earlier

out_dir = CONFIG["out_dir"]; os.makedirs(out_dir, exist_ok=True)
res_m = int(CONFIG.get("grid_resolution_m", 3000))
out_epsg = int(CONFIG.get("crs_epsg", 4326))
boundary_path = CONFIG.get("ca_boundary_path")

# 1) Persist boundary path back to CONFIG 
if not boundary_path or not os.path.exists(boundary_path):
    raise FileNotFoundError("CONFIG['ca_boundary_path'] missing or invalid. Rebuild boundary.")
CONFIG["ca_boundary_path"] = boundary_path

config_runtime_path = os.path.join(out_dir, "config_runtime.json")
with open(config_runtime_path, "w") as f:
    json.dump(CONFIG, f, indent=2)
print("Saved:", config_runtime_path)

# 2) Ensure we have an EPSG:3310 version for spatial ops
if grid_gdf.crs is None:
    raise ValueError("grid_gdf has no CRS. Rebuild grid.")
grid_3310 = grid_gdf.to_crs(3310) if grid_gdf.crs.to_epsg() != 3310 else grid_gdf

# 3) Save operational GeoParquet in 3310 + lightweight WGS84 preview
parquet_3310 = os.path.join(out_dir, f"grid_{res_m}m_CA_epsg3310.parquet")
grid_3310.to_parquet(parquet_3310, index=False)
print("Saved:", parquet_3310, "| cells:", len(grid_3310))

# Optional small preview in 4326 for quick map checks
preview_4326 = grid_3310.to_crs(4326).head(500)  # cap to avoid huge files
geojson_preview = os.path.join(out_dir, f"grid_{res_m}m_CA_head500_epsg4326.geojson")
preview_4326.to_file(geojson_preview, driver="GeoJSON")
print("Saved:", geojson_preview)

# 4) Compute and save metadata
cell_area_km2 = (res_m/1000.0)**2
effective_land_km2 = float((grid_3310.get("land_frac", 1.0) * cell_area_km2).sum())
implied_cell_km = float((effective_land_km2 / len(grid_3310))**0.5)
minx, miny, maxx, maxy = grid_3310.total_bounds
bbox_km = ((maxx-minx)/1000.0, (maxy-miny)/1000.0)

meta = {
    "timestamp_utc": dt.utcnow().isoformat(timespec="seconds") + "Z",
    "grid_resolution_m": res_m,
    "crs_ops_epsg": 3310,
    "crs_export_default_epsg": out_epsg,
    "cells": int(len(grid_3310)),
    "effective_land_area_km2": round(effective_land_km2, 2),
    "implied_cell_km": round(implied_cell_km, 4),
    "bbox_km_width_height": [round(bbox_km[0], 2), round(bbox_km[1], 2)],
    "has_land_frac": bool("land_frac" in grid_3310.columns),
    "boundary_path": boundary_path,
    "parquet_3310_path": parquet_3310,
    "geojson_preview_4326_path": geojson_preview,
}

meta_path = os.path.join(out_dir, f"grid_{res_m}m_CA_meta.json")
with open(meta_path, "w") as f:
    json.dump(meta, f, indent=2)
print("Saved:", meta_path)
meta


Saved: /Users/Shared/blueleaflabs/hydropulse/results/config_runtime.json
Saved: /Users/Shared/blueleaflabs/hydropulse/results/grid_3000m_CA_epsg3310.parquet | cells: 46495
Saved: /Users/Shared/blueleaflabs/hydropulse/results/grid_3000m_CA_head500_epsg4326.geojson
Saved: /Users/Shared/blueleaflabs/hydropulse/results/grid_3000m_CA_meta.json


{'timestamp_utc': '2026-01-12T06:13:53Z',
 'grid_resolution_m': 3000,
 'crs_ops_epsg': 3310,
 'crs_export_default_epsg': 4326,
 'cells': 46495,
 'effective_land_area_km2': 410516.3,
 'implied_cell_km': 2.9714,
 'bbox_km_width_height': [np.float64(915.0), np.float64(1059.0)],
 'has_land_frac': True,
 'boundary_path': '/Users/Shared/blueleaflabs/hydropulse/results/california_boundary.gpkg',
 'parquet_3310_path': '/Users/Shared/blueleaflabs/hydropulse/results/grid_3000m_CA_epsg3310.parquet',
 'geojson_preview_4326_path': '/Users/Shared/blueleaflabs/hydropulse/results/grid_3000m_CA_head500_epsg4326.geojson'}